In [66]:
#Load necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from numpy.fft import fft, ifft
from IPython.display import Audio
from scipy import signal
from ipykernel import kernelapp as app
from numpy.linalg import inv

In [67]:
FRAME_SIZE = 1024

In [68]:
# An algorithm to detect whether a frame is voiced or not
def voice_decider(frame):
    isVoiced = 0
    energy = 0
    
    for i in range(len(frame)):
        energy = energy + frame[i]*frame[i]

    if(energy>990000000):
        isVoiced = 1
        
    return isVoiced

In [69]:
# Load contaminated signal by crowd noise
Fs, data = read('test_audio.wav')
numFrames = int(len(data) / 1024)
output = np.zeros((len(data),1))
window_num = 0
noise_spectrum = np.zeros(FRAME_SIZE)

mu = 10 ** (-12)
tmp = np.zeros(FRAME_SIZE)
X = np.zeros((FRAME_SIZE, FRAME_SIZE))

In [71]:
for i in range(numFrames):
    #print(lms_filter)
    frame = data[i * FRAME_SIZE : ((i+1) * FRAME_SIZE)]
    
    if(voice_decider(frame.astype(float)) == 0):
        curFft = np.abs(np.fft.fft(frame))
        # For unvoiced frames, collect statistics about noise spectrum
        noise_spectrum = (noise_spectrum * window_num + curFft)/(window_num + 1)
        window_num = window_num + 1
        
        # Attenuate noise
        output[i * FRAME_SIZE : ((i+1) * FRAME_SIZE),0] = frame/10000
        continue
        
    DFT = np.fft.fft(frame)
    
    for t in range(FRAME_SIZE):
        # Spectrum subtraction
        if (np.abs(DFT[t]) != 0):
            DFT[t] = (np.abs(DFT[t]) - noise_spectrum[t])*DFT[t]/np.abs(DFT[t])

    signal = np.fft.ifft(DFT)
    signal = np.real(signal)
    
    for j in range(1024):
        if (j==0):
            tmp[j] = frame[0]
        else:
            tmp[j] = frame[1024-j]
            
    for j in range(1024):
        X[j,:] = tmp
        tmp = np.roll(tmp, 1)
    
    output[i * FRAME_SIZE : ((i+1) * FRAME_SIZE), 0] = inv(X ** np.transpose(X)) ** X ** signal

/Users/lihongyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in power
/Users/lihongyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in power


ValueError: could not broadcast input array from shape (1024,1024) into shape (1024)

In [47]:
# Play out the original audio
Audio(data, rate=Fs)

In [56]:
# Play out the processed audio
Audio(output, rate=Fs)